In [50]:
# cargando bibliotecas requeridas para realizar análisis de sentimiento

import pyarrow as pa
import numpy as np
import pyarrow.parquet as pq
import pandas as pd
from textblob import TextBlob

In [51]:
# leyendo archivo

user_reviews = pq.read_table('user_reviews.parquet')

In [52]:
# Mostrar las primeras filas de cada DataFrame
print("Primeras y ultimas filas de user_reviews:")
user_reviews

Primeras y ultimas filas de user_reviews:


pyarrow.Table
user_id: string
user_url: string
funny: string
posted: string
last_edited: string
item_id: int64
helpful: string
recommend: bool
review: string
__index_level_0__: int64
----
user_id: [["76561197970982479","76561197970982479","76561197970982479","js41637","js41637",...,"76561198312638244","76561198312638244","LydiaMorley","LydiaMorley","LydiaMorley"]]
user_url: [["http://steamcommunity.com/profiles/76561197970982479","http://steamcommunity.com/profiles/76561197970982479","http://steamcommunity.com/profiles/76561197970982479","http://steamcommunity.com/id/js41637","http://steamcommunity.com/id/js41637",...,"http://steamcommunity.com/profiles/76561198312638244","http://steamcommunity.com/profiles/76561198312638244","http://steamcommunity.com/id/LydiaMorley","http://steamcommunity.com/id/LydiaMorley","http://steamcommunity.com/id/LydiaMorley"]]
funny: [["","","","","",...,"","","1 person found this review funny","",""]]
posted: [["Posted November 5, 2011.","Posted July 15, 20

In [53]:
# crear una función a la que llamar para analizar cada reseña y clasificarla dentro de los parámetros dados

def sentiment_analysis(text):
    
    analysis = TextBlob(str(text))
    score = analysis.sentiment.polarity

    if score > 0:
        return 2  # Positive
    elif score < 0:
        return 0  # Negative
    else:
        return 1  # Neutral


In [54]:
# Convirtiendo user_reviews en un dataframe de pandas

user_reviews_pandas = user_reviews.to_pandas()

In [55]:
# Aplicar la función de análisis de sentimiento a la columna "review" y crear la columna "sentiment_analysis"

user_reviews_pandas['sentiment_analysis'] = np.where(user_reviews_pandas['review'].notna(), user_reviews_pandas['review'].apply(lambda x: sentiment_analysis(x)), 1)

In [56]:
# Mostrar las primeras filas de cada DataFrame
print("Primeras y ultimas filas de user_reviews:")
user_reviews_pandas

Primeras y ultimas filas de user_reviews:


,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review,sentiment_analysis
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,2
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.,2
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,2
3,js41637,http://steamcommunity.com/id/js41637,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,2
4,js41637,http://steamcommunity.com/id/js41637,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...,0
...,...,...,...,...,...,...,...,...,...,...
59300,76561198312638244,http://steamcommunity.com/profiles/76561198312...,,Posted July 10.,,70,No ratings yet,True,a must have classic from steam definitely wort...,2
59301,76561198312638244,http://steamcommunity.com/profiles/76561198312...,,Posted July 8.,,362890,No ratings yet,True,this game is a perfect remake of the original ...,2
59302,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,1 person found this review funny,Posted July 3.,,273110,1 of 2 people (50%) found this review helpful,True,had so much fun plaing this and collecting res...,2
59303,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,,Posted July 20.,,730,No ratings yet,True,:D,2


In [57]:
# Verificar si hay algún valor no nulo en la columna 'review'
hay_valores_no_nulos = user_reviews_pandas['review'].isnull().any()

if hay_valores_no_nulos:
    print("Hay al menos un valor nulo en la columna 'review'.")
else:
    print("Ninguno de los valores en la columna 'review' son NaN.")


Ninguno de los valores en la columna 'review' son NaN.


In [58]:
# eliminar la columna "review"

user_reviews_pandas = user_reviews_pandas.drop('review', axis=1)

In [59]:
user_reviews_pyarrow = pa.Table.from_pandas(user_reviews_pandas)
pq.write_table(user_reviews_pyarrow, "user_reviews.parquet")


